In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from wordcloud import WordCloud

#to data preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

#NLP tools
import re
import nltk
import string
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

#train split and fit models
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

#model selection
from sklearn.metrics import confusion_matrix, accuracy_score

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sasha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv("Hatespeech.csv")
data

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [3]:
print(data. head())

   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  


In [4]:
data["labels"] = data["class"]. map({0: "Hate Speech", 1: "Offensive Speech", 2: "No Hate and Offensive Speech"})
data = data[["tweet", "labels"]]
print(data. head())

                                               tweet  \
0  !!! RT @mayasolovely: As a woman you shouldn't...   
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...   
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...   
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...   
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...   

                         labels  
0  No Hate and Offensive Speech  
1              Offensive Speech  
2              Offensive Speech  
3              Offensive Speech  
4              Offensive Speech  


In [5]:
stopword=set(stopwords.words('english'))
stemmer = nltk. SnowballStemmer("english")

In [6]:
def clean (text):
    text = str(text). lower()
    text = re.sub('[.?]', '', text)
    text = re.sub('https?://\S+|www.\S+', '', text)
    text = re.sub('<.?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w\d\w', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
data["tweet"] = data["tweet"].apply(clean)

C:\Users\Sasha\AppData\Local\Temp\ipykernel_13944\1670938275.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["tweet"] = data["tweet"].apply(clean)


In [7]:
data

,tweet,labels
0,rt mayasolov woman shouldnt complain clean ho...,No Hate and Offensive Speech
1,rt mlee boy dat coldtyga dwn bad cuffin dat h...,Offensive Speech
2,rt urkindofbrand dawg rt babif ever fuck bitc...,Offensive Speech
3,rt cganderson vivabas look like tranni,Offensive Speech
4,rt shenikarobert shit hear might true might f...,Offensive Speech
...,...,...
24778,yous muthafin lie 0lifeask earl coreyemanuel r...,Offensive Speech
24779,youv gone broke wrong heart babi drove redneck...,No Hate and Offensive Speech
24780,young buck wanna eat dat nigguh like aint fuck...,Offensive Speech
24781,youu got wild bitch tellin lie,Offensive Speech


In [8]:
x = np.array(data["tweet"])
y = np.array(data["labels"])
cv = CountVectorizer()
X = cv.fit_transform(x)

In [9]:
# sentiment_analyzer = VS()

# def count_tags(tweet_c):  
    
#     space_pattern = '\s+'
#     giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
#         '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
#     mention_regex = '@[\w\-]+'
#     hashtag_regex = '#[\w\-]+'
#     parsed_text = re.sub(space_pattern, ' ', tweet_c)
#     parsed_text = re.sub(giant_url_regex, 'URLHERE', parsed_text)
#     parsed_text = re.sub(mention_regex, 'MENTIONHERE', parsed_text)
#     parsed_text = re.sub(hashtag_regex, 'HASHTAGHERE', parsed_text)
#     return(parsed_text.count('URLHERE'),parsed_text.count('MENTIONHERE'),parsed_text.count('HASHTAGHERE'))

# def sentiment_analysis(tweet):   
#     sentiment = sentiment_analyzer.polarity_scores(tweet)    
#     twitter_objs = count_tags(tweet)
#     features = [sentiment['neg'], sentiment['pos'], sentiment['neu'], sentiment['compound'],twitter_objs[0], twitter_objs[1],
#                 twitter_objs[2]]
#     return features

# def sentiment_analysis_array(tweets):
#     features=[]
#     for t in tweets:
#         features.append(sentiment_analysis(t))
#     return np.array(features)

# sentiment = sentiment_analysis_array(data["tweet"])
# sentiment

In [10]:
# Sentiment = pd.DataFrame({'-ve':sentiment[:,0],'+ve':sentiment[:,1],'Neutral':sentiment[:,2],'Compound':sentiment[:,3],
#                             'url_tag':sentiment[:,4],'mention_tag':sentiment[:,5],'hash_tag':sentiment[:,6]})
# Sentiment

In [11]:
# X= pd.DataFrame(Sentiment)
# y = np.array(data["labels"])

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state =40)

In [13]:
lr = LogisticRegression()

lr.fit(X_train, y_train)

c:\Users\Sasha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [14]:
y_pred_lr = lr.predict(X_test)
cm = confusion_matrix(y_test, y_pred_lr)
print(cm)

[[  35    8   89]
 [   2  374   38]
 [  25   48 1860]]


In [15]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_lr)

0.9152884227511093

In [16]:
inp = input()
inp = cv.transform([inp]).toarray()
print(lr.predict(inp))

['Offensive Speech']


In [17]:
import pickle

In [18]:
pickle.dump(lr, open('model.pkl', 'wb'))